In [1]:
! pip install diffusers transformers accelerate scipy safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os
import PIL
import torch
from PIL import Image, ImageDraw
from diffusers import StableDiffusionInpaintPipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
content_dir = "/kaggle/input/photo-reconstruction/Dataset/Testing_Data/"
mask_dir = "/kaggle/working/mask_files"
gen_dir = "/kaggle/working/gen_files"

In [4]:
metadata = pd.read_csv(content_dir + "masked_info.csv")

In [5]:
metadata

,Unnamed: 0,filename,box1_row,box1_col,box2_row,box2_col
0,0,Tiger (1024).jpeg,58,27,104,15
1,1,Elephant-Train (1195).jpeg,92,38,98,94
2,2,Tiger (980).jpeg,12,116,42,33
3,3,Elephant-Train (1842).jpeg,2,77,174,174
4,4,Dog-Train (1753).jpeg,52,75,84,160
...,...,...,...,...,...,...
195,195,Elephant-Train (544).jpeg,138,53,42,131
196,196,Dog-Train (333).jpeg,81,48,139,168
197,197,Tiger (1851).jpeg,38,8,18,6
198,198,Dog-Train (135).jpg,160,29,140,104


In [6]:
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)
if not os.path.exists(gen_dir):
    os.makedirs(gen_dir)

In [7]:
def makemasks(content_dir, mask_dir, meta):
    for idx in range(meta.shape[0]):
        _, filename, mask1_y, mask1_x, mask2_y, mask2_x = meta.iloc[idx,:].tolist()
        
        file_loc = content_dir + "/" + filename
        
        maskshape_1 = [(mask1_x, mask1_y), (mask1_x + 75, mask1_y + 75)]
        maskshape_2 = [(mask2_x, mask2_y), (mask2_x + 75, mask2_y + 75)]
        
        baseimg = Image.new(mode = "1", size=(256,256))
        maskimg = ImageDraw.Draw(baseimg)
        maskimg.rectangle(maskshape_1, fill ="#FFFFFF")
        maskimg.rectangle(maskshape_2, fill ="#FFFFFF")
        
        maskname = mask_dir + "/" +filename[0:filename.find(".")] + '.jpg'
        baseimg.save(maskname)

In [8]:
makemasks(content_dir, mask_dir, metadata)

In [9]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
).to('cuda')

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/clip/feature_extraction_clip.py:31: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  FutureWarning,


In [10]:
for idx in range(metadata.shape[0]):
    filename = metadata['filename'][idx]
    
    img = Image.open(content_dir + filename)
    msk = Image.open(mask_dir + "/" +filename[0:filename.find(".")] + '.jpg')
    prmpt = 'picture of a ' + filename.split(' (')[0].split('-')[0]
    
    gen_image = pipe(prompt=prmpt, image=img, mask_image=msk, height = 256, width = 256).images[0]
    
    gen_image.save(gen_dir + "/" + filename[0:filename.find(".")] + '.jpg')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
test_path = '/kaggle/input/photo-reconstruction/Dataset/Testing_Data/'
def submission_data(test_path):
    test_mask_data = pd.read_csv(test_path + 'masked_info.csv').drop(['Unnamed: 0'],axis=1)
    submission = []
    for i in range(len(test_mask_data)):
        if  (i%10)==0:
            print(i)
        filename, y1, x1, y2, x2 = test_mask_data.loc[i,]
        mask = makemasks(filename, test_mask_data, metadata)
        mask = mask.resize((512,512))
        masked_image = Image.open(test_path + filename).resize((512,512))
        prompt = "animal"
        image = pipe(prompt= prompt, image=masked_image, mask_image=mask).images[0]
        image = image.resize((256, 256))
        im = np.array(image)
        b, g, r = cv.split(im)
        result = np.zeros((256, 256))
        normalized_b = cv.normalize(b,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        normalized_g = cv.normalize(g,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        normalized_r = cv.normalize(r,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        for i in range(y1 , y1+75):
            for j in range(x1, x1+75):
                temp1 = filename + '_' + 'box1' + '_' +  str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', normalized_b[i,j]))
                submission.append((temp1 + '1', normalized_g[i,j]))
                submission.append((temp1 + '2', normalized_r[i,j]))
        for i in range(y2 , y2+75):
            for j in range(x2, x2+75):
                temp1 = filename + '_' + 'box2' + '_' + str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', normalized_b[i,j]))
                submission.append((temp1 + '1', normalized_g[i,j]))
                submission.append((temp1 + '2', normalized_r[i,j]))
        df = pd.DataFrame(submission, columns=['filename_box_pixel','value'])
    return(df)

In [12]:
test_path = '/kaggle/input/photo-reconstruction/Dataset/Testing_Data/'
df = submission_data(test_path)

0


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U1')) -> None

In [ ]:
df

In [ ]:
import shutil
shutil.make_archive('/kaggle/working/gen_files', 'zip', '/kaggle/working/')